# Restaurant Performance & Menu Optimization Analysis

# Project Overview

This project analyzes a quarter’s worth of transactional order data from a fictitious restaurant serving international cuisine to evaluate menu performance, revenue drivers, and operational efficiency. The goal is to move beyond descriptive insights and deliver clear, data-backed recommendations that improve profitability and operational decision-making.

Using transaction-level order data, this analysis identifies underperforming menu items, high-value orders, peak and off-peak demand periods, and cuisine-level opportunities for menu optimization and staffing alignment.

# Business Problem

Restaurants often face margin pressure due to menu bloat, inefficient staffing, and misalignment between customer demand and operational resources. Without data-driven insights, decisions around menu changes and staffing levels are typically based on intuition rather than evidence.

This project addresses the following core questions:

Which menu items and categories truly drive value?

Where are resources being allocated inefficiently?

How can menu and staffing decisions be optimized to improve profitability?

# Project Objectives

1. Identify high- and low-performing menu items based on order volume and revenue contribution.

2. Analyze category- and cuisine-level performance to uncover demand patterns.

3. Examine high-spend orders to understand purchasing behavior and revenue concentration.

4. Evaluate temporal ordering trends to identify peak and off-peak periods.

5. Assess operational efficiency by aligning order volume with staffing demand.

6. Provide actionable recommendations for menu optimization and staffing adjustments.